In [8]:
import pandas as pd
from ib_insync import *
from random import randint
from datetime import datetime, timezone, timedelta
import numpy as np
pd.options.mode.chained_assignment = None 

In [9]:
data = pd.read_csv('EUR_110.csv')
data['sign'] = np.where(data['type']=='Short', -1,1)
data['entryTime'] = data['entryTime'].astype('datetime64[ns]')
data['entryTime'] = pd.to_datetime(data['entryTime'])
data['entryTime_time']  = data.entryTime.dt.time
data['exitTime'] = data['exitTime'].astype('datetime64[ns]')
data['exitTime'] = pd.to_datetime(data['exitTime'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tradeN          2606 non-null   int64         
 1   symbol          2606 non-null   object        
 2   type            2606 non-null   object        
 3   shares          2606 non-null   int64         
 4   entrySignal     2606 non-null   object        
 5   entryTime       2606 non-null   datetime64[ns]
 6   entryPrice      2606 non-null   float64       
 7   exitSignal      2606 non-null   object        
 8   exitTime        2606 non-null   datetime64[ns]
 9   exitPrice       2606 non-null   float64       
 10  profit          2606 non-null   float64       
 11  runUp           2606 non-null   float64       
 12  drawDown        2606 non-null   float64       
 13  sign            2606 non-null   int64         
 14  entryTime_time  2606 non-null   object        
dtypes: d

In [10]:
'''Interactive Brokers Conection'''

util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 4001, clientId= randint(0, 30), timeout=30)

<IB connected to 127.0.0.1:4001 clientId=10>

In [49]:

# port 7496 for live TWS - 7497 for paper TWS
# port 4001 for live TWS - 4001 for Ib gateway

def download_data(symbol, day):
    contract = Forex(symbol)
    end_date = day
    bars = ib.reqHistoricalData(
        contract,
        endDateTime=(end_date), 
        durationStr='2 M',
        barSizeSetting='15 mins',
        whatToShow='MIDPOINT',
        
        useRTH=True
        )

    if bars is not None:
        return bars

def make_df(bars, symbol):
    # convert to pandas dataframe:
    df = util.df(bars)
    df.rename(columns={'date':'time'}, inplace=True)
    df.set_index('time', inplace=True)
    df['Instrument'] = symbol[:3] +  "/" + symbol[3:]
    #all_prices = pd.concat([all_prices, df], axis = 0)
    df.index = pd.to_datetime(df.index)
    return df

def create_trade_df(all_data, entryTime,exitTime, row, name):
    #df.to_excel('missing rows' + name+ str(i+1)+'.xlsx')
    trade_df = all_data[(all_data.index >= entryTime) & (all_data.index <= exitTime)]
    trade_df['sign'] = data.loc[row,'sign']
    #print('test create_trade')
    #print(trade_df)

    return trade_df

def pnl_calc(trade_df, data, entry, exit,i):
    trade_df.loc[entry, 'close'] = data.loc[i, 'entryPrice' ]
    trade_df.loc[exit, 'close'] = data.loc[i, 'exitPrice' ]
    trade_df['15min_pnl'] = (trade_df['close'].pct_change())*trade_df['sign']
    trade_df['trade_pnl_cum'] = trade_df['15min_pnl'].cumsum(0)
    trade_df.drop(columns=['volume','average','barCount'], inplace=True)
    return trade_df
        
def file_from_row(data, i, all_data):
    symbol = data.loc[i,'symbol'].replace('/', '' )
    day = data.loc[i, 'exitTime'].date()
    entryTime = data.loc[i, 'entryTime' ]
    entryTime = entryTime - timedelta(minutes=15)
    exitTime = data.loc[i, 'exitTime' ]
    #bars = download_data(symbol, day)
    #data_df = make_df(bars, symbol)
    #display(data_df)
    #print(entryTime)
    #print(exitTime)
    #print(type(data_df.index))
    trade_df = create_trade_df(all_data, entryTime,exitTime, i, symbol)
    #display(trade_df)
    trade_df = pnl_calc(trade_df, data, entryTime, exitTime,i)
    #display(trade_df)
    trade_df['ind'] = trade_df.index.shift(periods=1,freq='15min')
    trade_df['minutes'] = (trade_df['ind'] - trade_df['ind'].shift()).dt.total_seconds()\
    .cumsum().div(60).fillna(0).astype(int)
    trade_df.set_index(['ind'], inplace=True)
    trade_df['time'] = trade_df.index
    trade_df['time'] = trade_df['time'].dt.time
    #trade_df.to_excel(symbol + '_row' + str(i+1)+'_NY.xlsx')
    return trade_df

def find_max(df):
    max_row = df['trade_pnl_cum'].idxmax()
    max_min = df.loc[max_row,'minutes']
    return int(max_min)
    # Convert the percentage strings to percentage numbers.


In [33]:
days = ['20221020 00:00:00','20220820 00:00:00','20220620 00:00:00',
        '20220420 00:00:00','20220220 00:00:00','20220120 00:00:00',
        '20211020 00:00:00','20210620 00:00:00','20210220 00:00:00',
        '20201020 00:00:00','20200620 00:00:00','20200220 00:00:00',
        '20191020 00:00:00','20190920 00:00:00','20190220 00:00:00',
        '20181020 00:00:00','20180620 00:00:00','20180220 00:00:00',]


In [34]:
symbol = 'EURUSD'
all_data = pd.DataFrame()
for day in days:
    print(day)
    all_bars = download_data(symbol, day)
    data_months = make_df(all_bars, symbol)
    all_data = pd.concat([all_data,data_months],axis=0)
    display(all_data)
all_data

20221020 00:00:00


,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2022-10-19 22:45:00,0.976045,0.976665,0.976010,0.976545,-1.0,-1.0,-1,EUR/USD
2022-10-19 23:00:00,0.976545,0.976660,0.975675,0.975760,-1.0,-1.0,-1,EUR/USD
2022-10-19 23:15:00,0.975760,0.976015,0.975710,0.975805,-1.0,-1.0,-1,EUR/USD


20220820 00:00:00


,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2022-08-19 15:45:00,1.003850,1.004075,1.003565,1.004020,-1.0,-1.0,-1,EUR/USD
2022-08-19 16:00:00,1.004020,1.004330,1.003925,1.004085,-1.0,-1.0,-1,EUR/USD
2022-08-19 16:15:00,1.004085,1.004270,1.003980,1.004150,-1.0,-1.0,-1,EUR/USD


20220620 00:00:00


...

20180620 00:00:00


,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2018-06-19 22:45:00,1.158395,1.158625,1.157735,1.158085,-1.0,-1.0,-1,EUR/USD
2018-06-19 23:00:00,1.158085,1.158170,1.157825,1.157915,-1.0,-1.0,-1,EUR/USD
2018-06-19 23:15:00,1.157915,1.158000,1.157685,1.157975,-1.0,-1.0,-1,EUR/USD


20180220 00:00:00


,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2018-02-19 22:45:00,1.238205,1.238595,1.237935,1.238585,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:00:00,1.238585,1.238860,1.238230,1.238860,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:15:00,1.238860,1.239345,1.238630,1.238840,-1.0,-1.0,-1,EUR/USD


,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2018-02-19 22:45:00,1.238205,1.238595,1.237935,1.238585,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:00:00,1.238585,1.238860,1.238230,1.238860,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:15:00,1.238860,1.239345,1.238630,1.238840,-1.0,-1.0,-1,EUR/USD


In [44]:
all_data

,open,high,low,close,volume,average,barCount,Instrument
time,,,,,,,,
2022-08-21 17:15:00,1.004270,1.004270,1.004145,1.004195,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:30:00,1.004195,1.004220,1.003995,1.004040,-1.0,-1.0,-1,EUR/USD
2022-08-21 17:45:00,1.004040,1.004070,1.003970,1.004015,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:00:00,1.004015,1.004045,1.003295,1.003950,-1.0,-1.0,-1,EUR/USD
2022-08-21 18:15:00,1.003950,1.004080,1.003535,1.003615,-1.0,-1.0,-1,EUR/USD
...,...,...,...,...,...,...,...,...
2018-02-19 22:45:00,1.238205,1.238595,1.237935,1.238585,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:00:00,1.238585,1.238860,1.238230,1.238860,-1.0,-1.0,-1,EUR/USD
2018-02-19 23:15:00,1.238860,1.239345,1.238630,1.238840,-1.0,-1.0,-1,EUR/USD


In [50]:
trades_dict = {}
for i in data.index:
    df = file_from_row(data, i, all_data)
    trade = 'tradeN_' + str(i+1)
    trades_dict[trade] = df


In [51]:
entrytimes = data.entryTime.dt.time.unique()

In [53]:
buckets = pd.DataFrame(index = entrytimes)
x = 0
for trade in trades_dict.keys():
    df = trades_dict[trade]
    if len(df)<3:
        continue
    
    for entrytime in entrytimes:
        print(entrytime)
        
        name = entrytime.strftime('%H:%M:%S')
        if entrytime == df.time[0]:
            print (entrytime)
            #print(df.time[0])
            #print(entrytime.strftime('%H:%M:%S'))
            #print(type(name))
            try :
                buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
           # buckets.loc[entrytime,'row'+ str(x+1)]=buckets.loc[entrytime,'row'+ str(x+1)].astype(int)
            except:
                print(df)

            x+=1

      # print(name)          


08:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
0

/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
0

/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
1

...

08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
1

/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
2

16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
21:45:00
21:30:00
10:00:00
12:15:00
09:45:00
04:45:00
08:30:00
18:15:00
12:45:00
00:00:00
23:45:00
06:45:00
16:45:00
00:45:00
20:45:00
11:15:00
07:45:00
23:15:00
13:00:00
05:00:00
11:30:00
17:45:00
14:00:00
21:15:00
02:00:00
22:15:00
12:00:00
00:15:00
04:15:00
13:45:00
17:15:00
15:15:00
13:30:00
22:00:00
16:59:00
10:30:00
15:30:00
00:30:00
16:00:00
20:00:00
18:30:00
20:15:00
19:15:00
16:15:00
19:30:00
19:45:00
23:30:00
15:45:00
18:00:00
22:45:00
08:45:00
07:30:00
21:00:00
20:30:00
02:30:00
02:15:00
22:30:00
09:15:00
01:15:00
07:15:00
10:15:00
03:30:00
03:00:00
06:00:00
06:30:00
09:30:00
04:30:00
07:00:00
14:15:00
14:45:00
02:45:00
03:15:00
04:00:00
01:00:00
05:30:00
14:30:00
15:00:00
01:45:00
03:45:00
16:30:00
17:30:00
10:45:00
11:45:00
06:15:00
13:15:00
05:45:00
09:00:00
12:30:00
08:15:00
08:00:00
19:00:00
23:00:00
01:30:00
05:15:00
11:00:00
2

/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  buckets.loc[entrytime,'row'+ str(x+1)]= int(find_max(df))
/var/folders/4p/2cf471312vg1pjmqdwnxxk6w0000gn/T/ipykernel_16785/3201836703.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [54]:
buckets

,row1,row2,row3,row4,row5,row6,row7,row8,row9,row10,...,row1475,row1476,row1477,row1478,row1479,row1480,row1481,row1482,row1483,row1484
08:45:00,45.0,NaN,225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,375.0,375.0,375.0,NaN,NaN,NaN
07:30:00,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21:00:00,NaN,NaN,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20:30:00,NaN,NaN,NaN,NaN,135.0,135.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,465.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
average = buckets.mean(axis=1)
median = buckets.median(axis=1)
std = buckets.std(axis=1)

In [56]:
buckets['average'] = average
buckets['median'] = median
buckets['std'] = std

In [57]:
buckets

,row1,row2,row3,row4,row5,row6,row7,row8,row9,row10,...,row1478,row1479,row1480,row1481,row1482,row1483,row1484,average,median,std
08:45:00,45.0,NaN,225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,375.0,375.0,375.0,NaN,NaN,NaN,85.555556,45.0,111.742910
07:30:00,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.000000,75.0,104.303814
21:00:00,NaN,NaN,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.500000,345.0,305.896224
20:30:00,NaN,NaN,NaN,NaN,135.0,135.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.000000,105.0,262.543874
02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,465.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.750000,22.5,188.922862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.000000,30.0,12.549900
23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.500000,337.5,307.591450
15:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,525.000000,525.0,NaN


In [58]:
buckets['average_time'] = pd.to_datetime(buckets['average'], unit='m').dt.strftime('%H:%M')
buckets['median_time'] = pd.to_datetime(buckets['median'], unit='m').dt.strftime('%H:%M')
buckets['std_time'] = pd.to_datetime(buckets['std'], unit='m').dt.strftime('%H:%M')

In [59]:
buckets.sort_index().to_excel('trade_progression_2022_04_09.xlsx')